In [7]:
!pip install beautifulsoup4

Requirement not upgraded as not directly required: beautifulsoup4 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages


In [8]:
!pip install html5lib

Requirement not upgraded as not directly required: html5lib in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from html5lib)
Requirement not upgraded as not directly required: webencodings in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from html5lib)
Requirement not upgraded as not directly required: setuptools>=18.5 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from html5lib)


In [9]:
!pip install requests

Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests)
Requirement not upgraded as not directly required: urllib3<1.23,>=1.21.1 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests)
Requirement not upgraded as not directly required: certifi>=2017.4.17 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests)


#### Reading the page

In [10]:
from bs4 import BeautifulSoup
import requests

In [11]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [12]:
soup = BeautifulSoup(source,'lxml')

#### Find the tbody part it contains the PostCode, Borough and Neighborhood table

In [13]:
tb=soup.find('tbody')


#### Loop through the table body, get every row via TR tag

In [15]:
import pandas as pd

df = pd.DataFrame(columns=('PostCode', 'Borough', 'Neighborhood'))
df.head()

for row in tb.find_all('tr'):
    try:
        fields = row.find_all('td')
        postcode = fields[0].text.replace('\n','')
        borough = fields[1].text.replace('\n','')
        neighborhood = fields[2].text.replace('\n','')
    
    
    
        if borough != 'Not assigned':
            if neighborhood == 'Not assigned':
                neighborhood = borough           
        
        
            df=df.append(pd.DataFrame([[postcode,borough,neighborhood]],columns=('PostCode', 'Borough', 'Neighborhood')))
        
    except Exception as e:
            yt_link = None      
    

In [16]:
df.head(10)


,PostCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
0,M4A,North York,Victoria Village
0,M5A,Downtown Toronto,Harbourfront
0,M5A,Downtown Toronto,Regent Park
0,M6A,North York,Lawrence Heights
0,M6A,North York,Lawrence Manor
0,M7A,Queen's Park,Queen's Park
0,M9A,Etobicoke,Islington Avenue
0,M1B,Scarborough,Rouge
0,M1B,Scarborough,Malvern


In [17]:
df.shape


(211, 3)

#### Group by PostCode and Bourough and The Neighborhoood will be a list 

In [18]:
df2=df.groupby(['PostCode','Borough'])['Neighborhood'].apply(list)
df2.head(10)


PostCode  Borough    
M1B       Scarborough                                     [Rouge, Malvern]
M1C       Scarborough             [Highland Creek, Rouge Hill, Port Union]
M1E       Scarborough                  [Guildwood, Morningside, West Hill]
M1G       Scarborough                                             [Woburn]
M1H       Scarborough                                          [Cedarbrae]
M1J       Scarborough                                [Scarborough Village]
M1K       Scarborough        [East Birchmount Park, Ionview, Kennedy Park]
M1L       Scarborough                    [Clairlea, Golden Mile, Oakridge]
M1M       Scarborough    [Cliffcrest, Cliffside, Scarborough Village West]
M1N       Scarborough                        [Birch Cliff, Cliffside West]
Name: Neighborhood, dtype: object

In [24]:
df2.shape
df2 = df2.to_frame().reset_index()

In [20]:
!wget -O location.csv https://cocl.us/Geospatial_data

--2019-04-29 12:00:06--  https://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-04-29 12:00:07--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 185.235.236.197
Connecting to ibm.box.com (ibm.box.com)|185.235.236.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-04-29 12:00:07--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-04-2

In [22]:
loc_df = pd.read_csv('location.csv')
loc_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [25]:
result = pd.merge(df2,
                 loc_df,
                 left_on='PostCode',
                 right_on='Postal Code')
result.head()

,PostCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"[Rouge, Malvern]",M1B,43.806686,-79.194353
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]",M1C,43.784535,-79.160497
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]",M1E,43.763573,-79.188711
3,M1G,Scarborough,[Woburn],M1G,43.770992,-79.216917
4,M1H,Scarborough,[Cedarbrae],M1H,43.773136,-79.239476
